# (실습-7) Vector 유사도 검색 실습 - Pinecone

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 Pinecone를 이용하여 vector 유사도 검색을 실험해 본다. <br>
  Elasticsearch 및 Faiss와 비교하여 사용성이 어떻게 다른지 확인한다. <br>
2) 수강 목표
  * Pinecone를 이용하여 vector 유사도 검색을 할 수 있다.
  * Pinecone API를 사용할 수 있다.
  * Elasticsearch 및 Faiss 대비 차이점을 이해한다.

### 실습 목차
* 1. 데이터 전처리
* 2. 색인 및 검색

### 데이터셋 개요
* 데이터셋: wikimedia kowiki
* 데이터셋 개요 : wikimedia에서 제공하는 한국어 데이터셋

### 환경 설정
Pinecone-client를 설치한다. <br>
Sentence-transformers 패키지를 설치한다.

In [1]:
# Pinecone client 설치
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 1.4 MB/s eta 0:00:00


In [2]:
# 임베딩 생성을 위한 벡터 인코더 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 452.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4e4604a97c056032ecbbe565f6f7d0b28af21eed0406a612622725ad01dfb12c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


# 데이터 전처리

In [3]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
!python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result

--2024-01-24 06:57:17--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74589104 (71M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’

kowiki-latest-pages 100%[===================>]  71.13M  3.01MB/s    in 22s     

2024-01-24 06:57:39 (3.25 MB/s) - ‘kowiki-latest-pages-articles1.xml-p1p82407.bz2’ saved [74589104/74589104]

Cloning into 'wikiextractor'...
remote: Enumerating objects: 771, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 771 (delta 17), reused 21 (delta 14), pack-reused 741
Receiving objects: 100% (771/771), 1.31 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (450/450), done.
INFO: Pre

In [4]:
import json
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("hunkim/sentence-transformer-klue")

def get_embedding(sentences):
    # 입력 문장을 인코딩하여 임베딩을 얻음
    return model.encode(sentences)

wiki_dump_json_file = '/content/extract_result/AA/wiki_00'
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장
index_docs = []

for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # text와 title에 대한 임베딩을 계산하여 해당 필드에 추가
    json_data['embeddings_title'] = get_embedding(json_data['title']).tolist()
    json_data['embeddings_text'] = get_embedding(json_data['text']).tolist()

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [5]:
print(index_docs[1])

{'id': '9', 'revid': '36193661', 'url': 'https://ko.wikipedia.org/wiki?curid=9', 'title': '수학', 'text': '수학(數學, : mathematics, 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 매스매틱스(mathematics)라는 단어는 \'아는 모든 것\', \'배우는 모든 것\'이라는 뜻의 고대 그리스어 \'máthēma\'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.\n역사.\n역사적으로 고대부터 현대에 이르기까지 문명에 필수적인 건축, 천문학, 정치, 상업 등에 수학적 개념들이 응용되어 왔다. 교역·분배·과세 등 인류의 사회 생활에 필요한 모든 계산에 수학이 관여해 왔고, 농경 생활에 필수적인 천문 관측과 달력의 제정, 토지의 측량 또한 수학이 직접적으로 사용된 분야이다. 고대 수학을 크게 발전시킨 문명으로는 메소포타미아, 이집트, 인

In [6]:
# Pinecone 에서는 임베딩 이외의 필드를 딕셔너리 포맷(아래 metadata)으로 같이 넣어 줄 수 있음

index_docs_title = []
index_docs_text = []

for item in index_docs:
    index_docs_title.append({"id": item['id'], "metadata": {"title": item["title"], "text": item["text"][:200], "revid": int(item["revid"])},
                             "values": item["embeddings_title"]})
    index_docs_text.append({"id": item['id'], "metadata": {"title": item["title"], "text": item["text"][:200], "revid": int(item["revid"])},
                            "values": item["embeddings_text"]})

print(index_docs_title[0])

{'id': '5', 'metadata': {'title': '지미 카터', 'text': '제임스 얼 카터 주니어(: James Earl Carter, Jr., 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 ', 'revid': 36264616}, 'values': [-0.4782995283603668, -0.4664808511734009, -0.7922960519790649, -0.11245226114988327, 0.40639427304267883, 0.04852394759654999, 0.017006956040859222, -0.05078084021806717, 0.06199687719345093, 0.624271810054779, 0.046244170516729355, 0.15192341804504395, -0.421668142080307, -0.10195668041706085, -0.39047569036483765, -0.40188920497894287, -0.33431217074394226, -0.2537304162979126, 0.270402729511261, -0.3730441629886627, -0.3276699483394623, -0.4953809976577759, -0.4298953413963318, -0.073356032371521, -0.04655551165342331, -0.4938548803329468, 0.10923902690410614, -0.7731769680976868, 0.33000892400741577, 0.19122986495494843, 0.6521812081336975, -0.18116465210914612, 0.30834874510765076, 0.550782740116

# 색인 및 검색

In [7]:
# pinecone client 생성

from pinecone import Pinecone, PodSpec

# https://app.pinecone.io 에서 sign-up 한 뒤에 key 발급 작업 완료 후 생성된 key 사용
# 아래 api_key 교체 필요
pc = Pinecone(api_key="")

Pinecone에서 지원하는 세가지 Index 타입중 무료로 사용할 수 있는 "stater index" 사용 <br>
Index type에 대한 더 자세한 내용은 https://docs.pinecone.io/docs/manage-indexes 참고

In [8]:
# Starter index를 사용하여 index를 생성

# 이미 만들어둔 index가 있다면 아래 코드와 같이 삭제 가능
pc.delete_index("quickstart")

pc.create_index(name="quickstart", dimension=768, metric="euclidean",
                spec=PodSpec(environment="gcp-starter"))
pc.describe_index("quickstart")

{'dimension': 768,
 'host': 'quickstart-z139fja.svc.gcp-starter.pinecone.io',
 'metric': 'euclidean',
 'name': 'quickstart',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [9]:
# 문서를 색인

index = pc.Index("quickstart")

# 문서를 namespace에 나누어서 색인할 수 있음, 명시하지 않으면 최초 upsert 명령시 자동 생성
index.upsert(index_docs_title, namespace='title')
index.upsert(index_docs_text, namespace='text')

# 반영되는데 약간의 시간이 걸림
import time
time.sleep(20)

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.00122,
 'namespaces': {'text': {'vector_count': 61}, 'title': {'vector_count': 61}},
 'total_vector_count': 122}

In [10]:
# 검색 결과 확인, include_metadata가 True인 경우는 다른 필드들이 같이 출력됨

query_str = "문재인의 친구"
query_emb = get_embedding([query_str]).tolist()

# Index의 namespace에서 독립적으로 검색 수행
topk = index.query(namespace="title", vector=query_emb[0],
                   top_k=5, include_metadata=True)

# Metric이 euclidean인 경우는 점수가 낮을수록 유사도가 더 높음
topk

{'matches': [{'id': '85',
              'metadata': {'revid': 368112.0,
                           'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 '
                                   '제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 '
                                   '제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 '
                                   '장관을 역임했다.\n'
                                   '본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 '
                                   '막노동에 뛰어들었다가 독학으로 1975년 3월 30세',
                           'title': '노무현'},
              'score': 134.663834,
              'values': []},
             {'id': '102',
              'metadata': {'revid': 942.0, 'text': '', 'title': '음악인'},
              'score': 169.789139,
              'values': []},
             {'id': '77',
              'metadata': {'revid': 36036.0,
                           'text': '아미노산(: amino acid)은 아미노기(생물학적 조건에서 양성자화된 '
                                   '−NH3+ 형태), 카복실기(생물학적 조건에서 탈양성자화된 −COO− '


In [11]:
# 다른 질의로 한번더 실행

query_str = "대한민국 16대 대통령이 누구야?"
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="title", vector=query_emb[0],
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '34',
              'metadata': {'revid': 36250640.0,
                           'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 '
                                   '대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. '
                                   '제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n'
                                   '16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 '
                                   '한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 '
                                   '해양수',
                           'title': '대한민국 제16대 대통령 선거'},
              'score': 58.7418823,
              'values': []},
             {'id': '41',
              'metadata': {'revid': 36115896.0,
                           'text': '5월 31일은 그레고리력으로 151번째(윤년일 경우 152번째) 날에 '
                                   '해당한다.',
                           'title': '5월 31일'},
              'score': 178.818,
              'values': []},
             {'id': '40',
              'metadata': {'revid': 36114135.0

In [12]:
# text 필드에서 검색

query_str = "문재인의 친구"
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="text", vector=query_emb[0],
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '85',
              'metadata': {'revid': 368112.0,
                           'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 '
                                   '제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 '
                                   '제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 '
                                   '장관을 역임했다.\n'
                                   '본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 '
                                   '막노동에 뛰어들었다가 독학으로 1975년 3월 30세',
                           'title': '노무현'},
              'score': 160.842209,
              'values': []},
             {'id': '80',
              'metadata': {'revid': 699465.0,
                           'text': '토마스 만(: Thomas Mann, 1875년 6월 6일 ~ 1955년 '
                                   '8월 12일)은 독일의 평론가이자 소설가이다. 사상적인 깊이, 높은 식견, '
                                   '연마된 언어 표현, 짜임새 있는 구성 등에 있어서 20세기 독일 제일의 '
                                   '작가로 알려져 있다. 1929년 노벨 문학상을 비롯, 괴테 상 등 많은 상을 '


In [13]:
# 다른 질의로 한번더 실행

query_str = "대한민국 16대 대통령이 누구야?"
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="text", vector=query_emb[0],
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '34',
              'metadata': {'revid': 36250640.0,
                           'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 '
                                   '대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. '
                                   '제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n'
                                   '16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 '
                                   '한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 '
                                   '해양수',
                           'title': '대한민국 제16대 대통령 선거'},
              'score': 152.641815,
              'values': []},
             {'id': '85',
              'metadata': {'revid': 368112.0,
                           'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 '
                                   '제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 '
                                   '제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 '
                                   '장관을 역임했다.\n'
                     

In [14]:
# 오탈자 확인

query_str = "대한민국 16대 대통려이 누구야?"
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="text", vector=query_emb[0],
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '34',
              'metadata': {'revid': 36250640.0,
                           'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 '
                                   '대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. '
                                   '제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n'
                                   '16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 '
                                   '한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 '
                                   '해양수',
                           'title': '대한민국 제16대 대통령 선거'},
              'score': 151.257462,
              'values': []},
             {'id': '85',
              'metadata': {'revid': 368112.0,
                           'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 '
                                   '제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 '
                                   '제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 '
                                   '장관을 역임했다.\n'
                     

In [15]:
# 메타데이터로 필터링 하는 예시 1

query_str = "문재인의 친구"
filter={"revid": {"$gte": 300000}}
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="title", vector=query_emb[0], filter=filter,
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '85',
              'metadata': {'revid': 368112.0,
                           'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 '
                                   '제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 '
                                   '제13·15대 국회의원직을 역임했고, 김대중 정부에서 제6대 해양수산부 '
                                   '장관을 역임했다.\n'
                                   '본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 '
                                   '막노동에 뛰어들었다가 독학으로 1975년 3월 30세',
                           'title': '노무현'},
              'score': 134.663834,
              'values': []},
             {'id': '100',
              'metadata': {'revid': 783409.0,
                           'text': '수론에서의 뫼비우스 반전 공식(Möbius inversion '
                                   'formula)은 19세기 수학자 아우구스트 페르디난트 뫼비우스의 이름을 딴 '
                                   '공식이다.\n'
                                   '공식.\n'
                                   '"g"("n") 과 "f"("n")이 수론적 함수(arithmetic '

In [16]:
# 메타데이터로 필터링 하는 예시 2

query_str = "문재인의 친구"
filter={"revid": {"$ne": 368112}}
query_emb = get_embedding([query_str]).tolist()
topk = index.query(namespace="title", vector=query_emb[0], filter=filter,
                   top_k=5, include_metadata=True)
topk

{'matches': [{'id': '102',
              'metadata': {'revid': 942.0, 'text': '', 'title': '음악인'},
              'score': 169.789139,
              'values': []},
             {'id': '77',
              'metadata': {'revid': 36036.0,
                           'text': '아미노산(: amino acid)은 아미노기(생물학적 조건에서 양성자화된 '
                                   '−NH3+ 형태), 카복실기(생물학적 조건에서 탈양성자화된 −COO− '
                                   '형태), 특정한 곁사슬(R기)를 가지고 있는 유기 화합물이다. 모든 '
                                   '아미노산에 존재하는 원소는 탄소(C), 수소(H), 산소(O), '
                                   '질소(N)이다. 또한 황(S)은 시스테인과 메티오닌의 곁사슬에 존재하고, '
                                   '셀레늄(S',
                           'title': '아미노산'},
              'score': 178.54863,
              'values': []},
             {'id': '60',
              'metadata': {'revid': 20227.0, 'text': '', 'title': '새천년 민주당'},
              'score': 178.908737,
              'values': []},
             {'id': '20',
              'metadata': {'revid':

#Reference

## Required Package

sentence_transformers=2.2.2 <br>
pinecone-client==3.0.0
